In [6]:
import numpy as np
import mc
import math
import matplotlib.pyplot as plt
import joypy
from matplotlib import cm

In [7]:
Data_folder='/Users/xpsy1114/Documents/projects/multiple_clocks/data/ephys_recordings_200423/' 

mouse_recday='me11_05122021_06122021' #mouse a
a_rewards_configs = np.load(Data_folder+'Task_data_'+ mouse_recday+'.npy')
a_no_task_configs = len(a_rewards_configs)
a_locations = list()
a_neurons = list()
a_timings = list()
for session in range(0, a_no_task_configs):
    a_locations.append(np.load(Data_folder+'Location_raw_'+mouse_recday+'_'+str(session)+'.npy'))
    a_neurons.append(np.load(Data_folder+'Neuron_raw_'+mouse_recday+'_'+str(session)+'.npy'))
    a_timings.append(np.load(Data_folder+'trialtimes_'+mouse_recday+'_'+str(session)+'.npy'))

In [8]:
contrast_matrix = ((1,0,0), (1,-1,0), (1,0,-1), (0,1,0), (-1,1,0), (0,1,-1),(0,0,1), (-1,0,1), (0,-1,1) )
contrast_m = np.array(contrast_matrix)
coefficient = list()
contrast_results_all = list()

In [10]:
# this is just copy-pasted from the single_sub_ephy.py module; and adjusted to mouse_a.
for task_no, task_config in enumerate(a_rewards_configs):
    # first convert trial times from ms to bin number to match neuron and location arrays 
    # (1 bin = 25ms)
    timings_task = a_timings[task_no]
    for r, row in enumerate(timings_task):
        for c, element in enumerate(row):
            timings_task[r,c] = element/25

    # second, change locations and rewards to 0 and ignoring bridges
    locations_task = a_locations[task_no]
    for i, field in enumerate(locations_task):
        if field > 9: 
            locations_task[i] = locations_task[i-1]
        if math.isnan(field):
            # keep the location bc of timebins
            locations_task[i] = locations_task[i-1]


    # important: fields need to be between 0 and 8, and keep them as integers!
    locations_task = [int((field_no-1)) for field_no in locations_task]
    task_config = [int((field_no-1)) for field_no in task_config]

    # start of the second loop: loop through every single trial for this specific task config.
    coefficients_per_trial = np.zeros((len(timings_task),3))
    for trial_no, row in enumerate(timings_task):

        # current data of this specific run
        trajectory = locations_task[row[0]:row[-1]]


        curr_neurons = a_neurons[task_no][:,row[0]:row[-1]]
        # ISSUE 21.04.23:
        # if there are ONLY 0 for one timestep, the np.corrcoef will output nan for that instance. Maybe better:
        # replace by super super low value
        for col_no, column in enumerate(curr_neurons.T):
            if np.all(column == 0):
                curr_neurons[:,col_no] = 0.00001

        # some pre-processing to create my models.
        # to count subpaths
        subpath_file = [locations_task[row[0]:row[1]+1], locations_task[row[1]+1:row[2]+1], locations_task[row[2]+1:row[3]+1], locations_task[row[3]+1:row[4]+1]]
        timings_curr_run = [(elem - row[0]) for elem in row]

        # to find out the step number per subpath
        step_number = [0,0,0,0] 
        for path_no, subpath in enumerate(subpath_file):
            for i, field in enumerate(subpath):
                if i == 0:
                    count = 0
                elif field != subpath[i-1]:
                    count+=1
            step_number[path_no] = count

        # mark where steps are made
        for field_no, field in enumerate(trajectory):
            if field_no == 0:
                index_make_step = [0]
            elif field != trajectory[field_no-1]:
                index_make_step.append(field_no)


        location_model = mc.simulation.predictions.set_location_raw_ephys(trajectory, step_time = 1, grid_size=3, plotting = False, field_no_given= 1)
        midnight_model, clocks_model = mc.simulation.predictions.set_clocks_raw_ephys(trajectory, timings_curr_run, index_make_step, step_number, field_no_given= 1, plotting=False)

        # now create the model RDMs
        RSM_location = mc.simulation.RDMs.within_task_RDM(location_model, plotting = False, titlestring = 'Location RDM')
        RSM_clock = mc.simulation.RDMs.within_task_RDM(clocks_model, plotting = False, titlestring = 'Clock RDM')
        RSM_midnight = mc.simulation.RDMs.within_task_RDM(midnight_model, plotting = False, titlestring = 'Midnight RDM')

        # now create the data RDM
        # I am wondering if this is correct, though - maybe should I select those neurons where I know fit my predictions?
        RSM_neurons = mc.simulation.RDMs.within_task_RDM(curr_neurons, plotting = False, titlestring = 'Data RDM')

        # Lastly, create a linear regression with RSM_loc,clock and midnight as regressors and data to be predicted
        results_reg, scipy_regression_results = mc.simulation.RDMs.lin_reg_RDMs(RSM_neurons, regressor_one_matrix=RSM_clock, regressor_two_matrix= RSM_midnight, regressor_three_matrix= RSM_location)
        # print(f" The beta for the clocks model is {reg_res.coef_[0]}, for the midnight model is {reg_res.coef_[1]}, and for the location model is {reg_res.coef_[2]}")
        coefficients_per_trial[trial_no] = results_reg.coef_

In [ ]:
# now take the script from above and generate average-data RDMs per task config. 
# Maybe generate 1 for the first 4 trials, and one for the rest (learning and knowing) 
# this is just copy-pasted from the single_sub_ephy.py module; and adjusted to mouse_a.

for task_no, task_config in enumerate(a_rewards_configs):
    # first convert trial times from ms to bin number to match neuron and location arrays 
    # (1 bin = 25ms)
    timings_task = a_timings[task_no]
    for r, row in enumerate(timings_task):
        for c, element in enumerate(row):
            timings_task[r,c] = element/25

    # second, change locations and rewards to 0 and ignoring bridges
    locations_task = a_locations[task_no]
    for i, field in enumerate(locations_task):
        if field > 9: 
            locations_task[i] = locations_task[i-1]
        if math.isnan(field):
            # keep the location bc of timebins
            locations_task[i] = locations_task[i-1]


    # important: fields need to be between 0 and 8, and keep them as integers!
    locations_task = [int((field_no-1)) for field_no in locations_task]
    task_config = [int((field_no-1)) for field_no in task_config]

    # start of the second loop: loop through every single trial for this specific task config.
    
    for trial_no, row in enumerate(timings_task[0:4]):

        # current data of this specific run
        trajectory = locations_task[row[0]:row[-1]]


        curr_neurons = a_neurons[task_no][:,row[0]:row[-1]]
        # ISSUE 21.04.23:
        # if there are ONLY 0 for one timestep, the np.corrcoef will output nan for that instance. Maybe better:
        # replace by super super low value
        for col_no, column in enumerate(curr_neurons.T):
            if np.all(column == 0):
                curr_neurons[:,col_no] = 0.00001
        
        # store!
        
    # and then average!
    
    # now create the data RDM
    # I am wondering if this is correct, though - maybe should I select those neurons where I know fit my predictions?
    RSM_neurons = mc.simulation.RDMs.within_task_RDM(curr_neurons, plotting = True, titlestring = 'Data RDM')


